# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv



In [2]:
import dask.dataframe as dd

c:\Users\srees\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
parquet_files




['../../05_src/data/prices\\AAPL\\AAPL_2000\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2000\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.1.parquet',
 '../../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [48]:
# Write your code below.
# Read parquet files using the dask dataframe and set Ticker as Index
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
# dd_px

# Add lag for Close and Adj Close variables
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),AdjClose_lag_1 = x['Adj Close'].shift(1))
)
# dd_shift

# Add returns based on Close and the hi_lo_range and assign to dd_feat
dd_feat = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low'],
    
)

dd_feat



C:\Users\srees\AppData\Local\Temp\ipykernel_13268\3909774340.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,AdjClose_lag_1,Returns,hi_lo_range
npartitions=64,,,,,,,,,,,,
AAPL,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
ACN,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [49]:
# Write your code below.
# compute the dask dataframe 
pd_feat = dd_feat.compute()
# type(pd_feat)
#pd_feat['Moving_avg_returns']['ZBRA']

# calculate the rolling average on returns and assign the column to the pandas feature dataframe
pd_feat = pd_feat.assign(
    Moving_avg_returns = lambda x: x['Returns'].rolling(10).mean()
)

pd_feat



,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,AdjClose_lag_1,Returns,hi_lo_range,Moving_avg_returns
Ticker,,,,,,,,,,,,,
AAPL,2000-01-03 00:00:00+00:00,0.843076,0.999442,1.004464,0.907924,0.936384,535796800.0,2000,NaN,NaN,NaN,0.096540,NaN
AAPL,2000-01-04 00:00:00+00:00,0.771997,0.915179,0.987723,0.903460,0.966518,512377600.0,2000,0.999442,0.843076,-0.084310,0.084263,NaN
AAPL,2000-01-05 00:00:00+00:00,0.783293,0.928571,0.987165,0.919643,0.926339,778321600.0,2000,0.915179,0.771997,0.014633,0.067522,NaN
AAPL,2000-01-06 00:00:00+00:00,0.715508,0.848214,0.955357,0.848214,0.947545,767972800.0,2000,0.928571,0.783293,-0.086538,0.107143,NaN
AAPL,2000-01-07 00:00:00+00:00,0.749401,0.888393,0.901786,0.852679,0.861607,460734400.0,2000,0.848214,0.715508,0.047369,0.049107,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,2025-01-17 00:00:00+00:00,405.709991,405.709991,407.290009,402.290009,406.040009,270600.0,2025,402.720001,402.720001,0.007424,5.000000,0.005706
ZBRA,2025-01-21 00:00:00+00:00,418.070007,418.070007,419.850006,407.619995,407.619995,446000.0,2025,405.709991,405.709991,0.030465,12.230011,0.006642
ZBRA,2025-01-22 00:00:00+00:00,420.570007,420.570007,427.760010,419.589996,425.239990,497500.0,2025,418.070007,418.070007,0.005980,8.170013,0.006354


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

    It was not necessary to convert to pandas as we could have calculated the moving average returns by the similar way as in pandas.

+ Would it have been better to do it in Dask? Why?

    It would have been better to do it in Dask because in that case we dont need to compute which is a  time and resource intense operation.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.